<a href="https://colab.research.google.com/github/JulTob/SQL/blob/master/Lovelace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
%%writefile hello.adb
with Ada.Text_IO;
use Ada.Text_IO;

procedure Hello is
begin
   Put_Line ("Hello, Ada in Colab!");
end Hello;

Overwriting hello.adb


In [88]:
print('Compiling Ada code...')
!gnatmake hello.adb
print('Ada code compiled.')

Compiling Ada code...
x86_64-linux-gnu-gcc-10 -c hello.adb
x86_64-linux-gnu-gnatbind-10 -x hello.ali
x86_64-linux-gnu-gnatlink-10 hello.ali
Ada code compiled.


In [89]:
print('Running Ada executable:')
!./hello

Running Ada executable:
Hello, Ada in Colab!


In [90]:
%%writefile writer.ads
with Ada.Strings.Unbounded;

package Writer is
    -------------
    -- subtype Name is Ada.Strings.Unbounded.Unbounded_String;
    ------------
    subtype Text is Ada.Strings.Unbounded.Unbounded_String;
    ------------
    end Writer;

Overwriting writer.ads


In [91]:
%%writefile relations.ads
-- relation.ads

with Storage;
with Ada.Strings.Unbounded;
with Writer;
use Writer;

generic type Datum is private;

package Relations is

    type Relation is private;

    function New_Relation
                (Name  : String)

                return Relation;

    procedure Append
                (R : in out Relation;
                D : in Datum);

    function Element
                (R : Relation;
                Index : Positive)

                return Datum;

    function Length(
                R : Relation)

                return Natural;

    function Get_Name(
                R : Relation)

                return String;

    procedure Iterate
            (R       : Relation;
            Process : not null access procedure (D : Datum)
            );

    function Element
            (R     : Relation;
            Index : Positive)

            return Datum;

private

    package Data_Storage is new Storage (Datum);

    type Relation is record
        Rel_Name : Name;
        Data     : Data_Storage.Handle;
        end record;

end Relations;

Overwriting relations.ads


In [92]:
%%writefile relations.adb

with Ada.Strings.Unbounded;
use  Ada.Strings.Unbounded;

package body Relations is

    function New_Relation
        (Name : String)

        return Relation is
            begin
            R.Name := To_Unbounded_String(Name);
            Data_Storage.Init(R.Data);
            return R;
            end New_Relation;

    procedure Append(
        R : in out  Relation;
        D : in      Datum)
        is
            begin
            Data_Storage.Append (R.Data, D);
            end Append;

    function Length(R : Relation)
        return Natural is
            begin
            return Natural(Data_Storage.Length(R.Data));
            end Length;

    function Element
        (R : Relation;
        Index : Positive)

        return Datum
        is  begin
            return Data_Storage.Element_At(R.Data, Index);
            end Element;

    function Get_Name(
            R : Relation)

            return String
            is  begin
                return To_String(R.Name);
                end Get_Name;

    procedure Iterate
        (R       : Relation;
        Process : not null access procedure (D : Datum))
        is  begin
            for I in 1 .. Data_Storage.Length (R.Data) loop
                Process (Data_Storage.Element_At (R.Data, I));
                end loop;
            end Iterate;

end Relations;

Overwriting relations.adb


In [93]:
%%writefile storage.ads
-- storage.ads

with Ada.Containers.Indefinite_Vectors;

generic type Element is private;

package Storage is

   type Handle is limited private;

   procedure Init(S : out Handle);
   procedure Append(S : in out Handle; E : Element);

   function Element_At(S : Handle; Index : Positive) return Element;
   function Length(S : Handle) return Natural;

private

   package Vec is new Ada.Containers.Indefinite_Vectors
        (Index_Type => Positive,
        Element_Type => Element);

    type Handle is record
        V : Vec.Vector;
        end record;

end Storage;

Overwriting storage.ads


In [94]:
%%writefile storage.adb

-- storage.adb
package body Storage is

    procedure Init(S : out Handle) is
            begin
            S.V := Vec.Empty_Vector;
            end Init;

    procedure Append(S : in out Handle; E : Element) is
        begin
        S.V.Append(E);
        end Append;

    function Element_At(S : Handle; Index : Positive) return Element is
        begin
        return S.V.Element(Index);
        end Element_At;

    function Length(S : Handle) return Natural is
        begin
        return Natural(S.V.Length);
        end Length;

end Storage;

Overwriting storage.adb


In [95]:
%%writefile main.adb
with Ada.Text_IO;           use Ada.Text_IO;
with Ada.Strings.Unbounded; use Ada.Strings.Unbounded;
with Writer;                use Writer;

with Relations;

procedure Main is

    type Person is record
        Full_Name : Writer.Text;
        Age  : Natural;
        end record;

    package Persons is new Relations (Person);
    use Persons;

    R : Persons.Relation := Persons.New_Relation("People");

    procedure Print_Person (P : Person) is
        begin
        Put_Line (To_String (P.Full_Name) & " Age: " & P.Age'Image);
        end Print_Person;

        begin
        Append(R, ( Full_Name => To_Unbounded_String("Alice"),
                    Age => 30));
        Append(R, (Full_Name => To_Unbounded_String("Bob"),
                    Age => 42));

        Iterate (R, Print_Person'Access);

        end Main;

Overwriting main.adb


In [96]:
!gnatmake main.adb
!./main

x86_64-linux-gnu-gcc-10 -c main.adb
relations.ads:44:14: "Element" conflicts with declaration at line 23
relations.ads:54:10: completion of nonlimited type cannot be limited
relations.ads:54:10: component "Data" of type "Relation" has limited type
relations.ads:55:20: "Name" is not visible
relations.ads:55:20: non-visible declaration at a-textio.ads:159
relations.ads:55:20: non-visible declaration at system.ads:46
gnatmake: "main.adb" compilation error
Alice Age:  30
Bob Age:  42


In [97]:
print('Compiling Ada programs...')
!gnatmake main.adb

Compiling Ada programs...
x86_64-linux-gnu-gcc-10 -c main.adb
relations.ads:44:14: "Element" conflicts with declaration at line 23
relations.ads:54:10: completion of nonlimited type cannot be limited
relations.ads:54:10: component "Data" of type "Relation" has limited type
relations.ads:55:20: "Name" is not visible
relations.ads:55:20: non-visible declaration at a-textio.ads:159
relations.ads:55:20: non-visible declaration at system.ads:46
gnatmake: "main.adb" compilation error
